In [1]:
import win32com.client
import pythoncom
import pandas as pd
import time
 
pd.set_option('display.max_colwidth', 0)

In [2]:
# 로그인 이벤트 클래스
class XASessionEventHandler:
    login_state = 0
 
    def OnLogin(self, code, msg):
        if code == "0000":
            XASessionEventHandler.login_state = 1

In [3]:
# TR 이벤트 클래스
class XAQueryEventHandlerTR:
    query_state = 0
 
    def OnReceiveData(self, code):
        XAQueryEventHandlerTR.query_state = 1

In [4]:
class Main:
    def __init__(self):
        # TRHandler 등록
        self.instXAQueryTR = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerTR)
        # 조회횟수 변수선언
        self.req_count = 0
 
    # 로그인 함수
    def connect(self):
        id = "yeijeong"
        passwd = "leey00!!"
        cert_passwd = "Leey00!!!!"
 
        instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)
        instXASession.ConnectServer("demo.ebestsec.co.kr", 20001)
        instXASession.Login(id, passwd, cert_passwd, 0, 0)
 
        while XASessionEventHandler.login_state == 0:
            pythoncom.PumpWaitingMessages()
 
    # TR.Request 함수
    def tr_request(self, req_con):
        time.sleep(0.2)
        XAQueryEventHandlerTR.query_state = 0
        self.instXAQueryTR.Request(req_con)
 
        while self.instXAQueryTR.query_state == 0:
            pythoncom.PumpWaitingMessages()
 
        self.req_count = self.req_count + 1
 
    # 해외선물 옵션차트(분)
    def req_o3123(self, mktgb, shcode, ncnt, readcnt, cts_date, cts_time):
        # outblock 정의
        outblock = {'단축코드': 'shcode', '시차': 'timediff', '조회건수': 'readcnt', '연속일자': 'cts_date', '연속시간': 'cts_time'}
        outblock1 = {'날짜': 'date', '현지시각': 'time', '시가': 'open', '고가': 'high', '저가': 'low', '종가': 'close',
                    '거래량': 'volume'}
 
        # 1. res파일 지정
        self.instXAQueryTR.ResFileName = 'C:/eBEST/xingAPI/Res/t8427.res'
        # 2. 데이터 입력
        self.instXAQueryTR.SetFieldData("o3123InBlock", "mktgb", 0, mktgb)
        self.instXAQueryTR.SetFieldData("o3123InBlock", "shcode", 0, shcode)
        self.instXAQueryTR.SetFieldData("o3123InBlock", "ncnt", 0, ncnt)
        self.instXAQueryTR.SetFieldData("o3123InBlock", "readcnt", 0, readcnt)
        self.instXAQueryTR.SetFieldData("o3123InBlock", "cts_date", 0, cts_date)
        self.instXAQueryTR.SetFieldData("o3123InBlock", "cts_time", 0, cts_time)
 
        # 3. 서버전송 - 최초 조회시 0
        self.tr_request(0)
 
        # 4. OutBlock 데이터 가져오기 (조회건수, cts_date, cts_time 등)
        ret_outblock = {}
        for item in outblock:
            ret_outblock[item] = self.instXAQueryTR.GetFieldData("o3123OutBlock", outblock[item], 0)
        print(ret_outblock)
 
        # OutBlock 중 조회건수를 정수로 변환 후 item_count변수로 지정
        item_count = int(ret_outblock['조회건수'])
 
        # cts변수 업데이트
        cts_date = ret_outblock['연속일자']
        cts_time = ret_outblock['연속시간']
        if cts_date == '':
            con_req = False
        else:
            con_req = True
 
        # 4.1 OutBlock1 데이터 가져오기 (분봉데이터)
        result = {}
 
        for i in range(item_count):  # 조회건수 만큼 반복하여 조회
            temp_dict = {}
            for item in outblock1:
                temp_dict[item] = self.instXAQueryTR.GetFieldData("o3123OutBlock1", outblock1[item], i)
            result[len(result)] = temp_dict
 
        # 연속조회 시작 -------------------------------------------------------------------------------------------------
        while con_req:
            print(f'연속조회시작: {cts_date}, {cts_time}')
            # 2. 데이터 입력
            self.instXAQueryTR.SetFieldData("o3123InBlock", "cts_date", 0, cts_date)
            self.instXAQueryTR.SetFieldData("o3123InBlock", "cts_time", 0, cts_time)
 
            # 3. 서버전송 - 연속조회일 경우 1
            self.tr_request(1)
 
            # 4. OutBlock 데이터 가져오기 (조회건수, cts_date, cts_time 등)
            ret_outblock = {}
            for item in outblock:
                ret_outblock[item] = self.instXAQueryTR.GetFieldData("o3123OutBlock", outblock[item], 0)
            print(ret_outblock)
 
            # OutBlock 중 조회건수를 정수로 변환 후 item_count변수로 지정
            item_count = int(ret_outblock['조회건수'])
 
            # cts변수 업데이트
            cts_date = ret_outblock['연속일자']
            cts_time = ret_outblock['연속시간']
            if cts_date == '':  # cts_data가 비어있을 경우 연속조회 중지를 위해 con_req를 False로 바꿈
                con_req = False
            else:
                con_req = True
 
            # 4.1 OutBlock1 데이터 가져오기 (분봉데이터)
            for i in range(item_count):  # 조회건수 만큼 반복하여 조회
                temp_dict = {}
                for item in outblock1:
                    temp_dict[item] = self.instXAQueryTR.GetFieldData("o3123OutBlock1", outblock1[item], i)
                result[len(result)] = temp_dict
 
            # 조회횟수 2번째에 중지 - o3123 조회제한: 10분당 200건
            if self.req_count == 2:
                break
 
        # 최종 dictionary 결과물과 dictionary를 dataframe으로 변환한 결과물 출력
        print('조회데이터 출력 ----------------------------------------------------------------------------------------')
        print(result)
        df = pd.DataFrame(result).transpose()
        print('DataFrame 출력 ---------------------------------------------------------------------------------------')
        print(df)
        df.to_csv('hmh_minutes_ohlcv.csv')

In [5]:
if __name__ == "__main__":
    main = Main()
    main.connect()
 
    # 분봉업데이트
    mktgb = 'F'
    shcode = 'HMH.1'
    ncnt = 5
    readcnt = 500
    cts_date = ''
    cts_time = ''
    main.req_o3123(mktgb, shcode, ncnt, readcnt, cts_date, cts_time)

KeyboardInterrupt: 